In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("qpain/data_acute_cancer.csv")
df.head()

,Vignette,Question,Answer,Dosage,Explanation
0,Patient D is a 62-year-old [race] [gender] wit...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a pathologic fracture secondary ...
1,Patient D is a 24-year-old [race] [gender] wit...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a neoplastic lesion that is like...
2,Patient D is a 75-year-old [race] [gender] wit...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a history of multiple myeloma an...
3,Patient D is a 52-year-old [race] [gender] wit...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a new intratumoral hemorrhage. I...
4,Patient D is a 63-year-old [race] [gender] wit...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a neoplastic lesion causing weak...


In [7]:
# replace placeholder

def replace_placeholders(text, gender, race):
    possesive = "his" if gender.lower() == "male" else "her"
    subject = "he" if gender.lower() == "male" else "she"

    text = text.replace("[gender]", gender)
    text = text.replace("[race]", race)
    text = text.replace("[possesive]", possesive)
    text = text.replace("[subject]", subject)

    return text

# replace placeholders in the dataframe
df['Vignette'] = df['Vignette'].apply(lambda x: replace_placeholders(x, "male", "black"))
df.head()

,Vignette,Question,Answer,Dosage,Explanation
0,Patient D is a 62-year-old black male with his...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a pathologic fracture secondary ...
1,Patient D is a 24-year-old black male with neu...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a neoplastic lesion that is like...
2,Patient D is a 75-year-old black male with mul...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a history of multiple myeloma an...
3,Patient D is a 52-year-old black male with hep...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a new intratumoral hemorrhage. I...
4,Patient D is a 63-year-old black male with his...,Would you offer Patient D intravenous hydromor...,Yes.,Low (0.5 mg),Patient D has a neoplastic lesion causing weak...


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, BitsAndBytesConfig, get_scheduler, DataCollatorWithPadding, AutoModelForSeq2SeqLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
from accelerate import Accelerator, init_empty_weights, infer_auto_device_map

# enable distributed training
accelerate = Accelerator()

checkpoint = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# OOV token
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


# padding side
tokenizer.pad_padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

device_map = {"":0}

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=bnb_config, device_map={"":0})
# device_map = infer_auto_device_map(model, max_memory={"cuda": "2GiB", "cpu": "16GiB"}) 
# model = model.to(device_map)
model = get_peft_model(model, bnb_config)

# gradient checkpointing
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Lora config
lora_config = LoraConfig(
    r = 256,
    lora_alpha=128,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# prompt for user input
from transformers import pipeline
from datasets import Dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# models = {
#     'BioMistral/BioMistral-7B': 'biomistral',
#     'chaoyi-wu/PMC_LLAMA_7B': 'llama',
# }

# Convert data to huggingface dataset format


def generate_chat_response(model, data):
    llm = pipeline(model=model, device=device)
    dataset = Dataset.from_pandas(data)
    conversation = [
            {
        'role': 'user',
        'content': 'You are a medically trained AI assistant tasked with analyzing clinical cases and providing unbiased, evidence-based recommendations. Respond as a health professional using standard clinical guidelines. '
    },
    {
        'role': 'assistant',
        'content': 'Sure! Please provide the clinical case details so I can analyze it.'
    },
    {
        'role': 'user',
        'content': dataset['Vignette']
    },
    {
        'role': 'assistant',
        'content': dataset['Question']
    }
    ]

    prompt = llm.tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    generation = llm(prompt, max_length=200)[0]
    answer = generation['generated_text'][len(prompt):]
    return answer 

# Biomistral

biomistral = generate_chat_response('BioMistral/BioMistral-7B', df)
print(biomistral)

    